## PoC парсера сайта skifmusic.ru

In [2]:
#начинаем с импорта библиотек
import requests
from requests.auth import HTTPProxyAuth # для проксей
from  bs4 import BeautifulSoup as bs
import csv
from urllib.request import urljoin
#from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
from collections import deque
import pandas as pd
from multiprocessing import Pool
from datetime import datetime

In [4]:
# Задаем глобальные параметры
headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'
          }
domen = 'https://skifmusic.ru'
url='https://skifmusic.ru/catalog/catalog-100'
file_product_links = 'skifmusic_product_links.csv'
file_products = 'skifmusic_products.csv'

Блок вспомогательных функций

In [ ]:
# Фунцкция сохранения ссылок на продуктовые страницы
def save_products (result):
    with open(file_product_links, 'w', encoding = 'utf-8') as outfile:
        spamwriter = csv.writer(outfile, lineterminator='\n')
        spamwriter.writerow(['Product_Link']) 
        for i in result:
            spamwriter.writerow([i])
            
# Функция загрузки файла со ссылками на целевые страницы
def open_csv(file_product_links):
    results=[]
    with open(file_product_links, 'r') as data:
        data.readline().rstrip()
        for line in data:
            results.append(line[:-1])
    return results

def save_product_info(products):
    with open(file_products, 'w', encoding = 'utf-8') as outfile:
        spamwriter = csv.DictWriter(outfile, fieldnames=get_field_name(products), lineterminator='\n')    
        spamwriter.writeheader() #записывет заголовки столбцов равные ключам словаря
        spamwriter.writerows(products)
        
# Вспом.функция для сбора названия полей с словарях с продуктами для формирования заголовка в CSV (Без сортировки)
def get_field_name(products):
    field_names = set()
    for i in products:
        for k in i:
            field_names.add(k)
    return field_names

def get_visited_urls():
    try:
        file = file_products
        if file:
            df = pd.read_csv(file, encoding = 'utf-8')
            urls_done = df['5url'].tolist()
            visited = set(urls_done)
            return visited, df
    except FileNotFoundError:
        df=pd.DataFrame()
        visited=set()
        return visited, df # если файла не существует, товернем путой сет и список

    
def add_previous_part(df):
    if df.shape[0] != 0:
        file = file_products
        df.to_csv(file, mode='a', index=False, encoding = 'utf-8', header=False)
    else:
        pass  

In [99]:
#  Функция пагинатора
def get_pagination(url, headers):
    session = requests.Session()
    r = session.get(url, headers=headers)
    pagi={} 
    if r.status_code == 200:
        soup = bs(r.content, 'lxml')
        h = soup.find('h1', class_='header-h1').text
        t1 = soup.find('li', class_='last')
        pg = t1.find('a', href=True)['href'].split('page')[1]
        pagi[h]=[url]
        for i in range(2,int(pg)+1):
                       ii = str(url)+str('/page')+str(i)
                       pagi[h].append(ii)
    return pagi

In [96]:
# Блок сбора ссылок на продуктовые категории
session = requests.Session()
r = session.get(url, headers=headers)
soup = bs(r.text, 'lxml')
links = []
a = soup.find('div', class_='header__categories-menu')
b = a.find('div', attrs={'data-items': True})['data-items']
c=b.split(',')
urls={}
urls['url']=['"\\/catalog\\/gitaryi-1"']
for i in range(0,1227):
    f = c[i]
    if f[0:5] == '"url"':
        g = c[i].split(':')
        if g[0] ==  '"url"':
                urls['url'].append(g[1])
category_links=[]
a = urls.get('url')
for i in a:
    s = str(i).replace('"','').replace('\\','')
    link=str(domen)+str(s)
    category_links.append(link)
cat_links=[]
for i in category_links:
    ii = i.split('?f-type')
    cat_links.append(ii[0])
cat_links=list(set(cat_links))
cat_links.remove('https://skifmusic.ru/brand')
cat_links.remove('https://skifmusic.ru/sales')
cat_links

['https://skifmusic.ru/catalog/korpusa-dlya-akustiki-452',
 'https://skifmusic.ru/catalog/gitarnoe-usilenie-i-pedali-effektov-726',
 'https://skifmusic.ru/catalog/tsifrovyie-miksheryi-447',
 'https://skifmusic.ru/catalog/sabvuferyi-329',
 'https://skifmusic.ru/catalog/tarelki-46/tarelki-orkestrovyie',
 'https://skifmusic.ru/catalog/elektrogitaryi-12',
 'https://skifmusic.ru/catalog/kombousiliteli-dlya-bas-gitar-136',
 'https://skifmusic.ru/catalog/chehlyi-i-keysyi-dlya-zvukovogo-oborudovaniya-292',
 'https://skifmusic.ru/catalog/treschotka-655',
 'https://skifmusic.ru/catalog/predusiliteli-dlya-bas-gitar-134',
 'https://skifmusic.ru/catalog/krepleniya-dlya-remnya-394',
 'https://skifmusic.ru/catalog/mikrofonyi-7',
 'https://skifmusic.ru/catalog/eufoniumyi-546',
 'https://skifmusic.ru/catalog/tsifrovyie-pianino-37/tsifrovyie-pianino-klassicheskie',
 'https://skifmusic.ru/catalog/saksofonyi-284',
 'https://skifmusic.ru/catalog/plastiki-dlya-barabanov-49/plastiki-dlya-barabana',
 'https:/

In [97]:
# Мини список для отработки пагинации
result= ['https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90',
        'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87']

In [100]:
#Запуск пагинатора
grid_links=[]
for i in result:
    ii = get_pagination(i, headers)
    grid_links.append(ii)

В этом кейсе у меня каждая пага относится к категории, т.к. сайт большой и свливать все пагианции в одну кучу не хочется

In [102]:
grid_links

[{'Электроакустические гитары': ['https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90',
   'https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90/page2',
   'https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90/page3',
   'https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90/page4',
   'https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90/page5',
   'https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90/page6',
   'https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90/page7']},
 {'Классические гитары': ['https://skifmusic.ru/catalog/klassicheskie-gitaryi-87',
   'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87/page2',
   'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87/page3',
   'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87/page4',
   'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87/page5',
   'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87/page6',
   'https://skifmusic.ru/catalog/klassicheskie-git

In [103]:
# Создадим Серию с названиями категорий. Это позволит нам задавать категории для парсинга
catalog=[]
rg=range(0,len(grid_links))
for i in range(0,len(grid_links)):
    for item in grid_links[i]:
        catalog.append(item)
cat=pd.Series(catalog, range(0,len(grid_links)))
cat

0    Электроакустические гитары
1           Классические гитары
dtype: object

Дальше можно заморочится через input запросить название категории для парсинга и прасить только её, но мне лень сейчас это писать

In [24]:
# А вот и цикл для перебора выбранных категорий и получения всех ссылок в каждой категории
for k in range(0,len(grid_links)):
    print(grid_links[k].get(str(cat[k])))

['https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90', 'https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90/page2', 'https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90/page3', 'https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90/page4', 'https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90/page5', 'https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90/page6', 'https://skifmusic.ru/catalog/elektroakusticheskie-gitaryi-90/page7']
['https://skifmusic.ru/catalog/klassicheskie-gitaryi-87', 'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87/page2', 'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87/page3', 'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87/page4', 'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87/page5', 'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87/page6', 'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87/page7', 'https://skifmusic.ru/catalog/klassicheskie-gitaryi-87/page8', 'https://skifmus

In [28]:
# Функция сбора ссылок на целевые страницы
def get_product_links(url, headers, domen):
    session = requests.Session()
    r = session.get(url, headers=headers)
    soup = bs(r.text, 'lxml')
    links = []
    urls=set()
    if url:
        ul = soup.find('div', class_='product-list').find_all('a', href=True)
        if ul:
            for u in ul:
                urls.add(u['href'])
            for i in urls:
                link  = urljoin(domen,i)
                links.append(link)
    return links

In [36]:
# Блок сбора продуктовых урлов с грида
temp_plinks=[]
for k in range(0,len(grid_links)):
    aa = (grid_links[k].get(str(cat[k])))
    for a in aa:
        gpl = get_product_links(a, headers, domen)
        temp_plinks.append(gpl)
#выпрямляем список продуктовых урлов    
product_links=[]
for sublist in temp_plinks:
    for item in sublist:
        product_links.append(item)   
save_products(product_links)
print('DONE!')

DONE!


In [5]:
# ВЕРСИЯ 2 С ПРОМЕЖУТОЧНЫМ СОХРАНЕНИЕМ
def get_info2(headers, file_product_links): 
    global df
    results = open_csv(file_product_links)
    queue = deque()
    queue.extend(results)
    visited = get_visited_urls()[0]
    df = get_visited_urls()[1]
    session = requests.Session()
    products = [] 
    while tqdm(queue, desc='Getting info from product page url', leave=False):   
        url = queue.popleft()
        if url in visited:
            continue # начнет цикл while заново     
        params={} # словарь для занесения параметров//можно в конце цикла все добавить как в кинопоиске без создания словаря
        r = session.get(url, headers=headers)
        soup = bs(r.text, 'lxml')
        if soup:
            visited.add(url) #добавляем в посещенные, только когда получим суп. т.е. страница доступна
            try:
                params['5url']=url
                ee = soup.find('li', class_='product-images-gallery-thumbnails__item js-product-images-gallery-thumbnails--item')
                e = ee.find('img', src=True)['src']
                params['6img']=e
                a=soup.find('h1', class_='header-h1').text.strip()
                if a:
                    params['1_title']=a
                l1=[]
                l2=[]
                dct={}
                b=soup.find('ul', class_='list-unstyled').find_all('span', class_='bold')
                c=soup.find('ul', class_='list-unstyled').find_all('span', class_='value')
                for i in b:
                    l1.append(i.text.strip())
                for k in c:
                    l2.append(k.text.strip())
                dct = dict(zip(l1,l2))
                params.update(dct)
                cc=soup.find('div', class_='mt-10 mb-20')
                c = cc.find('span', class_='bigger-170 text-bold').text.replace('\xa0','')
                if c:
                    params['3_price']=c
                d=soup.find('div', class_='green').text.strip()
                if d:
                    params['4_stock']=d.strip()
                products.append(params)
            except:
                products.append(params)
                print('Часть тегов не обнаружена. Двигаюсь дальше')
                pass
        else:
            print('Проблемы с супом')
            save_product_info(products) # промежуточное сохранение на случай если суп не обнаружен
            return False
    save_product_info(products) #  для сохранения в CSV
    add_previous_part(df) # перезапись предыдущих результатов в случае перезапуска парсера
    print('DONE!')

In [95]:
get_info2(headers, file_product_links)

DONE!
